In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F
import mmap
import random
import pickle
import argparse

parser = argparse.ArgumentParser(description='This is a demonstration program')

# Here we add an argument to the parser, specifying the expected type, a help message, etc.
# parser.add_argument('-batch_size', type=str, required=True, help='Please provide a batch_size')

# args = parser.parse_args()

# Now we can use the argument value in our program.
# print(f'batch size: {args.batch_size}')
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# batch_size = args.batch_size # to use the batch_size cmd arg -> python file_name.py -batch_size 32
batch_size = 32
block_size = 128
max_iters = 200
learning_rate = 3e-4
eval_iters = 100
n_embd = 384
n_head = 4
n_layer = 4
dropout = 0.2

print(device)

cuda


In [2]:
chars = ""
with open("openwebtext/vocab.txt", 'r', encoding='utf-8') as f:
        text = f.read()
        chars = sorted(list(set(text)))
        
vocab_size = len(chars)

In [3]:
string_to_int = { ch:i for i,ch in enumerate(chars) }
int_to_string = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

In [4]:
# memory map for using small snippets of text from a single file of any size
#Bu kod bloğu, belirtilen bir dosyadan rastgele bir metin bloğu alarak, bu bloğu modelin eğitimi için bir veri yığınına dönüştüren iki fonksiyon içerir: get_random_chunk ve get_batch.
def get_random_chunk(split):  #Bu fonksiyon, belirtilen split (train veya val) için rastgele bir metin bloğunu dosyadan alır.
    filename = "openwebtext/train_split.txt" if split == 'train' else "openwebtext/val_split.txt" #Dosya adı, split parametresine göre belirlenir.
    with open(filename, 'rb') as f:
        with mmap.mmap(f.fileno(), 0, access=mmap.ACCESS_READ) as mm:  #mmap modülü kullanılarak dosyanın belleğe haritalanması (memory-mapped) gerçekleştirilir. Bu, dosyanın tamamının belleğe yüklenmesine gerek olmadan belirli bir bölümüne erişim sağlar.
            # Determine the file size and a random position to start reading
            #Dosya boyutu ve okuma başlamak için rastgele bir konum belirlenir.
            file_size = len(mm) 
            start_pos = random.randint(0, (file_size) - block_size*batch_size)

            # Seek to the random position and read the block of text
            #Belirtilen başlangıç konumuna gidilir ve belirli bir boyuttaki metin bloğu okunur.
            mm.seek(start_pos)
            block = mm.read(block_size*batch_size-1)

            # Decode the block to a string, ignoring any invalid byte sequences
            #Okunan blok, 'utf-8' kodlaması kullanılarak bir stringe çevrilir ve geçersiz bayt dizilerini yoksayar.
            decoded_block = block.decode('utf-8', errors='ignore').replace('\r', '')
            
            # Train and test splits
            #Bu string, önce encode fonksiyonu ile sayılara dönüştürülür ve ardından bir PyTorch Tensor'üne dönüştürülür.
            data = torch.tensor(encode(decoded_block), dtype=torch.long)
            
    return data


def get_batch(split): #Bu fonksiyon, get_random_chunk fonksiyonunu kullanarak rastgele bir metin bloğunu alır ve bu bloğu modelin eğitimi için bir veri yığınına dönüştürür.
    data = get_random_chunk(split)  #get_random_chunk fonksiyonu çağrılarak rastgele bir metin bloğu alınır.
    ix = torch.randint(len(data) - block_size, (batch_size,))  #Metin bloğundan rastgele indeksler seçilerek, giriş ve çıkış tensorleri oluşturulur.
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    #Oluşturulan tensorler, cihaza (device) taşınır ve çıkış olarak döndürülür.
    x, y = x.to(device), y.to(device) 
    return x, y 

In [5]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [6]:
class Head(nn.Module):     #  SCALED DOT - PRODUCT ATTENTION
    """ one head of self-attention """
#Bu sınıfın amacı, self-attention mekanizmasını tanımlamak ve bu mekanizmanın başlık başına farklı öğrenilebilir ağırlıklar ile uygulanabilmesini sağlamaktır.

    def __init__(self, head_size):  #Head sınıfının başlatıcıdır. head_size parametresini alır ve içinde self-attention işlemi için gerekli olan katmanları (linear layers) ve diğer önemli bileşenleri tanımlar.
        super().__init__()
        #key, query, ve value linear katmanları, self-attention işleminde kullanılan anahtar (key), sorgu (query) ve değer (value) bilgilerini oluşturur.
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size))) #tril isimli buffer, alt üçgen matrisi temsil eder. Bu, attention ağırlıklarının üst üçgen kısmını maskeleyerek kullanılmıştır.

        self.dropout = nn.Dropout(dropout) #dropout katmanı, dropout işlemi uygulanarak ağırlıkların bir kısmının rastgele sıfırlanmasını sağlar.

    def forward(self, x):  #Bu fonksiyon, self-attention işlemini gerçekleştirir.
        # input of size (batch, time-step, channels)  (Girdi xin boyutları (batch, time-step, channels) şeklinde olmalıdır.)
        # output of size (batch, time-step, head size)
        B,T,C = x.shape
        #key, query, ve value bilgileri hesaplanır.
        k = self.key(x)   # (B,T,hs)
        q = self.query(x) # (B,T,hs)
        # compute attention scores ("affinities")
        #Attention ağırlıkları (wei) hesaplanır ve maskeleme işlemi uygulanır.
        wei = q @ k.transpose(-2,-1) * k.shape[-1]**-0.5 # (B, T, hs) @ (B, hs, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T) #Softmax fonksiyonu ile normalizasyon yapılır.
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,hs)
        #Ağırlıklar ile değerler çarpılarak ağırlıklı toplam (out) elde edilir.
        out = wei @ v # (B, T, T) @ (B, T, hs) -> (B, T, hs)
        return out

# [1, 0, 0]
# [1, 0.6, 0]
# [1, 0.6, 0.4]
    
class MultiHeadAttention(nn.Module):   # MULTI- HEAD ATTENTION
    """ multiple heads of self-attention in parallel """

#Bu sınıf, num_heads sayısı kadar farklı başlık ile self-attention işlemlerini gerçekleştiren bir çoklu başlıklı self-attention mekanizmasını temsil eder.

    def __init__(self, num_heads, head_size):  #Bu fonksiyon, MultiHeadAttention sınıfının başlatıcıdır. num_heads ve head_size parametrelerini alır ve içinde bir dizi başlığı (Head sınıfını) ve çıkış özellik boyutunu düzenlemek için bir projeksiyon katmanını (proj) tanımlar.
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])  #heads isimli nn.ModuleList, Head sınıfının belirtilen sayıda başlığı ile oluşturulmuş bir liste içerir.
        self.proj = nn.Linear(head_size * num_heads, n_embd)  #proj linear katmanı, birleştirilmiş başlık çıkışlarını istenen çıkış boyutuna dönüştürmek için kullanılır.
        self.dropout = nn.Dropout(dropout)  #dropout katmanı, ağırlıkların bir kısmının rastgele sıfırlanması için kullanılır.

    def forward(self, x):  #Bu fonksiyon, Multi-Head Attention mekanizmasının ileri geçişini tanımlar.
        #Her başlığın (Head) çıkışı ayrı ayrı hesaplanır ve birleştirilir (torch.cat ile).
        out = torch.cat([h(x) for h in self.heads], dim=-1) # (B, T, F) -> (B, T, [h1, h1, h1, h1, h2, h2, h2, h2, h3, h3, h3, h3])
        out = self.dropout(self.proj(out)) #Birleştirilmiş çıkış, projeksiyon katmanından geçirilir ve son olarak dropout işlemine tabi tutulur.
        return out
    

class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

#Bu sınıf, bir giriş vektörünü alır, içinde birkaç lineer katman ve aktivasyon fonksiyonu ile işler, ve son olarak bir çıkış vektörü üretir. 
#Bu tür bir besleme ileri ağı, modelin öğrenme yeteneğini artırmak ve karmaşıklığını artırmak için kullanılır.

    def __init__(self, n_embd):
        super().__init__()  #Bu fonksiyon, FeedForward sınıfının başlatıcıdır. n_embd parametresini alır ve içinde birkaç lineer katman, aktivasyon fonksiyonu (ReLU) ve dropout işlemi içeren bir öğrenme ağı tanımlar.
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd), #İlk lineer katman, giriş boyutunu 4 katına çıkartır.
            nn.ReLU(),  #ReLU aktivasyon fonksiyonu, gizli katmanın çıkışına uygulanır.
            nn.Linear(4 * n_embd, n_embd),   #İkinci lineer katman, çıkış boyutunu tekrar n_embd boyutuna düşürür.
            nn.Dropout(dropout),  #Dropout işlemi, ağırlıkların bir kısmını rastgele sıfırlar. (%20 belirlersek rastgele %20lik kısmını sıfırlar)
        )

    def forward(self, x):  #Bu fonksiyon, besleme ileri ağının ileri geçişini tanımlar.
        return self.net(x) #Girdi x, ağın üzerinden geçirilir ve çıkış elde edilir.
    
class Block(nn.Module):
    """ Transformer block: communication followed by computation """
#Bu sınıf bir Transformer bloğunu temsil eder. Transformer mimarisi, bir dizi tekrarlanan bloğun bir araya getirilmesiyle oluşur. 
#Her bir blok, self-attention mekanizmasını ve ardından bir besleme ileri (feedforward) ağını içerir.

    def __init__(self, n_embd, n_head):  # Bu fonksiyon, Block sınıfının başlatıcıdır. n_embd ve n_head parametrelerini alır ve içinde bir self-attention (sa) mekanizması, bir besleme ileri ağı (ffwd), ve iki adet Layer Normalizasyon (ln1 ve ln2) katmanını tanımlar.
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head  #head_size hesaplanır, bu da her bir self-attention başlığının boyutunu temsil eder.
        self.sa = MultiHeadAttention(n_head, head_size) #sa isimli self-attention mekanizması, MultiHeadAttention sınıfı ile oluşturulan bir nesnedir.
        self.ffwd = FeedFoward(n_embd) #ffwd isimli besleme ileri ağı, FeedForward sınıfı ile oluşturulan bir nesnedir.
        self.ln1 = nn.LayerNorm(n_embd) #ln1 ve ln2 isimli Layer Normalizasyon katmanları, self-attention ve besleme ileri ağı çıkışlarını normalize eder.
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):  #Bu fonksiyon, Transformer bloğunun ileri geçişini tanımlar.
        y = self.sa(x) #İlk olarak, self-attention mekanizması (sa) üzerinden geçirilir ve çıkış (y) elde edilir.
        x = self.ln1(x + y) #Layer Normalizasyon ve residual bağlantısı uygulanır (x + y).
        y = self.ffwd(x)  #Ardından, besleme ileri ağı (ffwd) üzerinden geçirilir.
        x = self.ln2(x + y)  #Yine Layer Normalizasyon ve residual bağlantısı uygulanır (x + y).
        return x  #Son olarak, çıkış elde edilir.
    
class GPTLanguageModel(nn.Module):
    def __init__(self, vocab_size):  #Bu fonksiyon, GPTLanguageModel sınıfının başlatıcıdır. vocab_size parametresini alır ve içinde bir dizi bileşeni tanımlar.
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)  #Kelime gömme (embedding) tablosu, giriş kelimelerini gömme işlemi için kullanılır.
        self.position_embedding_table = nn.Embedding(block_size, n_embd)  #Pozisyon gömme tablosu, girişin her bir kelimesinin pozisyonunu gömme işlemi için kullanılır.
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])  #Bir dizi Transformer bloğunu içeren bir blok (block) sekansıdır.
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm  #Final Layer Normalizasyon katmanı, çıkışın normalize edilmesi için kullanılır.
        self.lm_head = nn.Linear(n_embd, vocab_size)  #Lineer katman, modelin kelime seviyesinde dil üretmesi için kullanılır.
        
        
        self.apply(self._init_weights)  #apply fonksiyonu, modelin ağırlıklarını başlangıçta belirlenmiş bir şekilde başlatmak için kullanılır.

    def _init_weights(self, module):  #Bu fonksiyon, ağırlıkları başlangıçta belirli bir şekilde başlatmak için kullanılır.
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(self, index, targets=None):  # Bu fonksiyon, GPT dil modelinin ileri geçişini tanımlar.
        #Giriş kelime indeksleri (index) ve opsiyonel hedefler (targets) alır.
        B, T = index.shape
        
        
        # idx and targets are both (B,T) tensor of integers
        #Kelime gömme ve pozisyon gömme işlemleri uygulanır.
        tok_emb = self.token_embedding_table(index) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)  #bir dizi Transformer bloğu (blocks) üzerinden geçirilir.

        #Çıkış, final Layer Normalizasyon katmanından geçirilir ve lineer katman ile kelime seviyesinde dil üretimi yapılır.
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)  #logits değişkeni, her bir kelimenin vocabülerdeki olası sıralı olasılıklarını içeren bir çıkış tensorunu temsil eder.
        
        #Eğer hedefler (targets) belirtilmişse, çıkış üzerinden çapraz entropi kaybı hesaplanır.
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss
    
    def generate(self, index, max_new_tokens):  #Bu fonksiyon, modelin verilen bir başlangıç dizisi üzerinden yeni kelime önerileri üretmesini sağlar.
        # index is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):  # Bu döngü, belirtilen max_new_tokens sayısı kadar yeni kelime önerisi üretmek için kullanılır.
            # crop idx to the last block_size tokens
            index_cond = index[:, -block_size:]  #Giriş indekslerini (index) son block_size token ile sınırlayan bir işlem gerçekleştirilir.
            # get the predictions
            logits, loss = self.forward(index_cond)  #Giriş indekslerinden (index_cond) ileri geçiş yaparak modelin çıkışını ve olası kaybı (loss) elde eder.
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)  #Sadece en son zaman adımına odaklanarak çıkışın son zaman adımını alır. Böylece, modelin en son tahminlerine odaklanır.
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)  # Softmax fonksiyonunu kullanarak çıkış logitlerini olasılıklara dönüştürür.
            # sample from the distribution
            index_next = torch.multinomial(probs, num_samples=1) # (B, 1)  #Olasılıklar üzerinden örnek seçerek (multinomial kullanılarak) bir sonraki indeksi (index_next) belirler.
            # append sampled index to the running sequence
            index = torch.cat((index, index_next), dim=1) # (B, T+1)  #Seçilen indeksi, mevcut dizinin sonuna ekler, böylece yeni bir öneri oluşturulur.
        return index  #Oluşturulan yeni diziyi döndürür. Bu, başlangıç dizisi üzerinden modelin önerdiği yeni kelimelerin tamamını içerir.

model = GPTLanguageModel(vocab_size)
# print('loading model parameters...')
# with open('model-01.pkl', 'rb') as f:
#     model = pickle.load(f)
# print('loaded successfully!')
m = model.to(device)

In [12]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
    print(iter)
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f"step: {iter}, train loss: {losses['train']:.3f}, val loss: {losses['val']:.3f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())

with open('model-01.pkl', 'wb') as f:
    pickle.dump(model, f)
print('model saved')

0
step: 0, train loss: 2.461, val loss: 2.340
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
step: 100, train loss: 2.309, val loss: 2.443
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
2.4919517040252686
model saved


In [11]:
prompt = 'Hello! Can you see me?'
context = torch.tensor(encode(prompt), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context.unsqueeze(0), max_new_tokens=100)[0].tolist())
print(generated_chars)

Hello! Can you see me?Pr mes, k.

ThipredburthennixpVMMaico icSadu as t nst 90095back blllplorthinthotse0 r hapay hath u2o
